<a href="https://colab.research.google.com/github/cras-lab/OpenAPI/blob/main/%EA%B8%88%EA%B0%90%EC%9B%90_OpenDart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

금감원이 Open DART 공시시스템에서 삼성전자의 연결재무와 독립재무<BR>
값을 읽어 오는 예제

OpenDART 공시스템에서 값 가져오기
먼저 필요한 모듈을 임포트 한다.

In [1]:
import requests
import pandas as pd

자신의 OpenDART 키를 설정한다. 금융통계정보시스템 Key와는 다르니 주의하자.

In [2]:
# OpenDART API 키
api_key = 'a06ed5ecaee6fac7168de666b7912090bfd1af23'

삼성전자의 종목코드를 입력한다.

In [3]:
# 삼성전자의 종목 코드
corp_code = '00126380'  # 삼성전자의 법인 코드 (예시, 실제 코드가 다를 수 있음)

당기 순이익을 저장할 리스트를 선언한다.

In [5]:
# 당기순이익 정보를 저장할 리스트
net_incomes = []

읽어올 연도의 범위를 설정한다.

In [6]:
# 연도 범위 설정
years = range(2015, 2023)

API를 호출하여 실제 값을 읽어 온다.

In [7]:
for year in years:
    # OpenDART API URL
    url = f'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year={year}&reprt_code=11011'

    # API 요청 보내기
    response = requests.get(url)
    data = response.json()

    # 결과 확인 및 당기순이익 추출 (연결재무와 단독재무 순으로 나옴)
    if 'list' in data:
        for item in data['list']:
            if item['account_nm'] == '당기순이익':
                net_income = {
                    'year': year,
                    'net_income': item['thstrm_amount']
                }
                net_incomes.append(net_income)

읽어온 값을 데이터 프레임으로 변환한다.

In [8]:
# DataFrame으로 변환
df = pd.DataFrame(net_incomes)

읽어 온 값의 열을 분리하고, 각각 연결 및 단독 재무로 저장

In [9]:
# 각 연도별 첫 번째 행은 첫 번째 열로, 두 번째 행은 두 번째 열로 분기
df['net_income'] = df['net_income'].str.replace(',', '').astype(float)  # 콤마 제거 후 숫자로 변환

df_grouped = df.groupby('year').apply(lambda x: x['net_income'].values).apply(pd.Series)
df_grouped.columns = ['Consolidated FS', 'Separate FS']
df_grouped.index.name = 'year'

결과를 출력해 본다.

In [ ]:
# 결과 출력
print("\nTransformed DataFrame:")
print(df_grouped)

그래프를 그려본다.

In [ ]:
df_grouped.plot()